# Data Exploration

In [42]:
#orte = ["a","b","ca", "d"]

def build_pairs(orte):
    pairs = []
    for index, value in enumerate(orte):
        if index == len(orte) - 1:
            break
        for j in range(index+1, len(orte)):
            pairs.append([value, orte[j]])
    return pairs
pairs = build_pairs(orte)
pairs

[['Falkensee', 'Herne'],
 ['Falkensee', 'Schwu00E4bisch Hall'],
 ['Falkensee', 'Berlin-Pankow MechZB'],
 ['Falkensee', 'Detmold'],
 ['Falkensee', 'Winterberg'],
 ['Falkensee', 'Mainz'],
 ['Falkensee', 'Landshut'],
 ['Falkensee', 'Kumhausen'],
 ['Falkensee', 'Mu00FCnchen Riesenfeld MechZB'],
 ['Falkensee', 'Flu00F6rsheim am Main'],
 ['Falkensee', 'Maintal'],
 ['Falkensee', 'Offenbach am Main'],
 ['Falkensee', 'Berlin-Britz MechZB'],
 ['Falkensee', 'Gevelsberg'],
 ['Falkensee', 'Winsen'],
 ['Falkensee', 'Coesfeld'],
 ['Falkensee', 'Neuss MechZB'],
 ['Falkensee', 'Bietigheim-Bissingen'],
 ['Falkensee', 'Wolmirstedt'],
 ['Falkensee', 'Ku00FChlungsborn'],
 ['Falkensee', 'Neckarsulm'],
 ['Falkensee', 'Bochum-Herne MechZB'],
 ['Falkensee', 'Stralsund'],
 ['Falkensee', 'Potsdam'],
 ['Falkensee', 'Ku00F6ln'],
 ['Falkensee', 'Ratingen MechZB'],
 ['Falkensee', 'Stollberg /Erzgeb.'],
 ['Falkensee', 'Guben'],
 ['Falkensee', 'Konstanz'],
 ['Falkensee', 'Herzlake'],
 ['Falkensee', 'Neustadt an der We

In [43]:
def similarity(string1, string2):
    if string1 in string2 or string2 in string1:
        return 1
    else:
        return 0
similarity("Bochum", "Bochum-Herne MechZB")

1

In [44]:
similarity("aa", "bb")

0

In [45]:
for pair in pairs:
    sim = similarity(pair[0], pair[1])
    if pair[0] == pair[1]:
        break
    if sim == 1:
        print("Sim({},{})={}".format(pair[0], pair[1], sim))

Sim(Herne,Bochum-Herne MechZB)=1
Sim(Berlin-Pankow MechZB,Berlin)=1
Sim(Mainz,Mainz-Hechtsheim MechZB)=1
Sim(Mainz,Mainz-Kastel)=1
Sim(Mu00FCnchen Riesenfeld MechZB,Mu00FCnchen)=1
Sim(Berlin-Britz MechZB,Berlin)=1
Sim(Neuss MechZB,Neuss)=1
Sim(Bochum-Herne MechZB,Bochum)=1
Sim(Ku00F6ln,Ku00F6ln MechZB)=1
Sim(Augsburg MechZB,Augsburg)=1
Sim(Ru00FCsselsheim,Ru00FCsselsheim am Main)=1
Sim(Koblenz,BZ 56, Carl-Spaeter-Str. 2f, 56070 Koblenz)=1
Sim(Koblenz,Koblenz MechZB)=1
Sim(Bonn MechZB,Bonn)=1
Sim(Wuppertal,Wuppertal MechZB)=1
Sim(Mu00FCnchen,Garching b. Mu00FCnchen)=1
Sim(Mu00FCnchen,Mu00FCnchen Am Neubruch MechZB)=1
Sim(Mu00FCnchen,Mu00FCnchen City Logistik Center MechZB)=1
Sim(Mu00FCnchen,Mu00FCnchen Neuaubing MechZB)=1
Sim(Mu00FCnchen,Garching bei Mu00FCnchen)=1
Sim(Aschheim,Aschheim MechZB)=1
Sim(Erlangen,Erlangen MechZB)=1
Sim(Dortmund MechZB,Dortmund)=1
Sim(Paderborn MechZB,Paderborn)=1
Sim(Iserlohn MechZB,Iserlohn)=1
Sim(Hamm,Hamm MechZB)=1
Sim(Darmstadt,Darmstadt MechZB)=1
Sim(B

In [60]:
if sim == 1:
    if sim[0] > sim[1]:
        sim.replace(sim[0], sim[1]) 
    elif sim[1] > sim[0]:
        sim.replace(sim[1], sim[0])


## import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('baseline')

In [24]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('baseline')

In [35]:
def json_to_df(json_file=None):
    """ Reads a single JSON file containing the
    shipment information and converts it to a
    structured dataframe
    """
    import pandas as pd
    from pandas.io.json import json_normalize
    import numpy as np
    logger.debug("Converting to dataframe: {}".format(json_file))
    df = pd.read_json(json_file, dtype={"shipment_number": str})
    normalized_df = json_normalize(df['events'])
    df.drop(["events"], axis=1, inplace=True)
    df = pd.concat([df, normalized_df], axis=1)
    if "Die Sendung wurde elektronisch angeku00FCndigt. Sobald die Sendung von uns bearbeitet wurde, erhalten Sie weitere Informationen." == df.tail(1)["status"].item():
        logger.warning("Final delivery state is strange! Ignoring the file {}".format(json_file))
        return
    # Some files have multiple states. It does not make sense!
    if df[df["status"] == "Die Sendung wurde erfolgreich zugestellt."].shape[0] > 1:
        logger.warning("Multiple final states! Ignoring the file {}".format(json_file))
        return
    if df["status"].isnull().any():
        logger.warning("Status column has missing values! Ignoring the file {}".format(json_file))
        return
    if len(df["url"].unique()) > 1:
        logger.warning("Multiple URLs for a single shipment number! Something is wrong")
    if len(df["shipment_number"].unique()) > 1:
        logger.warning("Multiple shipment numbers in the same file. Something is wrong")
    df['crawltime'] = pd.to_datetime(df['crawltime'], utc=True)
    df['datum'] = pd.to_datetime(df['datum'], utc=True)
    if df["status"].isnull().values.any():
        logger.warning("Column status has missing values! ShipmentNumber: {}".format(df["shipment_number"].values[0]))
    final_event_array = np.zeros(len(df))
    final_event_array[-1] = 1
    df["is_final_event"] = final_event_array
    return df

In [26]:
def files_to_df(path='../data'):
    """ Convert all files to a single dataframe
    """
    import os
    import pandas as pd
    files = []
    for r, d, f in os.walk(path):
        for file in f:
            if '.json' in file:
                files.append(os.path.join(r, file))
    dfs = [json_to_df(json_file=file) for file in files]
    df = pd.concat(dfs, sort=False)
    return df

In [27]:
df = files_to_df(path='../data')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  


In [28]:
orte = list(df["ort"].dropna().values)

In [29]:
orte = list(set(orte))

In [30]:
orte

['Falkensee',
 'Herne',
 'Schwu00E4bisch Hall',
 'Berlin-Pankow MechZB',
 'Detmold',
 'Winterberg',
 'Mainz',
 'Landshut',
 'Kumhausen',
 'Mu00FCnchen Riesenfeld MechZB',
 'Flu00F6rsheim am Main',
 'Maintal',
 'Offenbach am Main',
 'Berlin-Britz MechZB',
 'Gevelsberg',
 'Winsen',
 'Coesfeld',
 'Neuss MechZB',
 'Bietigheim-Bissingen',
 'Wolmirstedt',
 'Ku00FChlungsborn',
 'Neckarsulm',
 'Bochum-Herne MechZB',
 'Stralsund',
 'Potsdam',
 'Ku00F6ln',
 'Ratingen MechZB',
 'Stollberg /Erzgeb.',
 'Guben',
 'Konstanz',
 'Herzlake',
 'Neustadt an der Weinstrau00DFe',
 'Herzogenaurach',
 'BARCELONA, Spanien',
 'Dinkelsbu00FChl',
 'Freiberg',
 'Kriftel',
 'Augsburg MechZB',
 'Gu00F6ttingen',
 'EAST MIDLANDS, Grou00DFbritannien',
 'Radefeld',
 'Lu00FCdinghausen',
 'Unterhaching',
 'Sittensen',
 'Ru00FCsselsheim',
 'Neubiberg',
 'Rheinfelden (Baden)',
 'Singen (Hohentwiel)',
 'Balingen',
 'Koblenz',
 'Bad Urach',
 'Grou00DFenlu00FCder',
 'Schriesheim',
 'Bonn MechZB',
 'Gremberghoven MechZB',
 'Zev

### Are there any missing values?

In [36]:
import pandas as pd
def count_missing_values(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
    return missing_value_df
missing_value_df = count_missing_values(df)

In [37]:
missing_value_df

,column_name,percent_missing
shipment_number,shipment_number,0.0
crawltime,crawltime,0.0
url,url,0.0
datum,datum,0.0
status,status,0.0
ort,ort,0.0
is_final_event,is_final_event,0.0


Column "ort" has many missing values. We fill them with "Unbekannt"

In [38]:
def fill_missing_values(df):
    """ Fills missing values of specific columns
    in the data frame with pre-defined values.
    """
    df['ort'].fillna("Unbekannt", inplace=True)
    logger.debug("Filled missing values for ort column")
    return df

In [39]:
df = fill_missing_values(df)

Check again if there are any missing values

In [40]:
count_missing_values(df)

,column_name,percent_missing
shipment_number,shipment_number,0.0
crawltime,crawltime,0.0
url,url,0.0
datum,datum,0.0
status,status,0.0
ort,ort,0.0
is_final_event,is_final_event,0.0


### Data Stats

In [41]:
df.shape

(143748, 7)

Number of unique shipment numbers

In [ ]:
len(df.shipment_number.unique())

### Investigate Data Types

In [ ]:
df.info()

Everything regarding data types seems to be fine! No need for conversion

### Prepare data for exploration and machine learning

- There are places where at least two may be in the same city
- To simplify the analysis, we delete the unnecessery suffixes from these places.

In [ ]:
pd.DataFrame(df["ort"].value_counts()).head(20)

- further we transform the current datetime-format to a discrete time-format, so that we can summarize our data in more descrete time groups 

In [ ]:
def discretize_time(df):
    import numpy as np
    import pandas as pd
    from datetime import datetime
    
    df['datum'] = pd.DatetimeIndex(df['datum']).astype(np.int64)/1000000
    
    return df

In [ ]:
df = discretize_time(df)

In [ ]:
pd.DataFrame(df["datum"]).head()

- Given the features "datum", "status" and "ort" we want to predict the next "datum" and hopefully also the next "status" and the next "ort"
- We create 3 new columns. "datum_next", "ort_next" and "status_next".

In [ ]:
def create_target_columns(df):
    import numpy as np 

    df['status_next'] = df.groupby(['shipment_number'])['status'].shift(-1)
    df['ort_next'] = df.groupby(['shipment_number'])['ort'].shift(-1)
    df['datum_next'] = df.groupby(['shipment_number'])['datum'].shift(-1)
    # we define values for the case the package is already delivered.
    df.loc[df['is_final_event'] == 1, 'status_next'] = "SuccessDelivery"
    df.loc[df['is_final_event'] == 1, 'ort_next'] = "SuccessDelivery"
    df.loc[df['is_final_event'] == 1, 'datum_next'] = np.datetime64('today')
    
    return df

In [ ]:
# Disable warnings
pd.set_option('mode.chained_assignment', None)
df = create_target_columns(df)

In [ ]:
df.head()

In [ ]:
df.info()

### Data Stats

In [ ]:
count_missing_values(df)

In [ ]:
df.shape

Number of unique shipment numbers

In [ ]:
len(df.shipment_number.unique())

In [ ]:
len(df.status.unique())

In [ ]:
len(df.ort.unique())

### How many different final delivery states exist?

In [ ]:
final_delivery_states = df.groupby(['shipment_number']).tail(1)[["status"]]["status"].unique()
print(final_delivery_states)

### How many different initial delivery states exist?

In [ ]:
initial_delivery_states = df.groupby(['shipment_number']).head(1)[["status"]]["status"].unique()
print(initial_delivery_states)

### Distribution of final delivery states

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.xticks(rotation='vertical')

df[df["status"].isin(final_delivery_states)]["status"].hist(figsize=(15,5))

### Distribution of initial states

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.xticks(rotation='vertical')

df[df["status"].isin(initial_delivery_states)]["status"].hist(figsize=(10,5))

### Distribution of all states

These are the most common states

In [ ]:
pd.DataFrame(df["status"].value_counts()).head(10)

These are the least common states. What we see here is that there are some states that are basically the same and the following pattern:

"Die Sendung liegt in der ........ zur Abholung bereit"

But the address is each time different. This leads to too many states, which also makes machine learning hard. We ignore the address part and convert all these states to a single one.

In [ ]:
pd.DataFrame(df["status"].value_counts()).tail(10)

In [ ]:
df.loc[(df['status'].str.startswith("Die Sendung liegt in der")) & (df['status'].str.endswith("zur Abholung bereit.")), "status"] = "Die Sendung liegt in der ... zur Abholung bereit." 
df.loc[(df['status'].str.startswith("Die Sendung liegt ab sofort in der Filiale")) & (df['status'].str.endswith("zur Abholung bereit.")), "status"] = "Die Sendung liegt ab sofort in dert Filiale ... zur Abholung bereit."
df.loc[(df['status'].str.startswith("Die Sendung wird zur Abholung in die")) & (df['status'].str.endswith("gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich.")), "status"] = "Die Sendung wird zur Abholung in die ... gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich." 
df.loc[(df['status'].str.startswith("Die Sendung wurde zur Abholung in die")) & (df['status'].str.endswith("gebracht.")), "status"] = "Die Sendung wurde zur Abholung in die ... gebracht." 
df.loc[(df['status'].str.startswith("Die Zustellung am gewu00FCnschten Liefertag")) & (df['status'].str.endswith("wurde gewu00E4hlt.")), "status"] = "Die Zustellung am gewu00FCnschten Liefertag ... wurde gewu00E4hlt." 

df.loc[(df['status_next'].str.startswith("Die Sendung liegt in der")) & (df['status_next'].str.endswith("zur Abholung bereit.")), "status_next"] = "Die Sendung liegt in der ... zur Abholung bereit." 
df.loc[(df['status'].str.startswith("Die Sendung liegt ab sofort in der Filiale")) & (df['status'].str.endswith("zur Abholung bereit.")), "status"] = "Die Sendung liegt ab sofort in dert Filiale ... zur Abholung bereit."
df.loc[(df['status_next'].str.startswith("Die Sendung wird zur Abholung in die")) & (df['status_next'].str.endswith("gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich.")), "status_next"] = "Die Sendung wird zur Abholung in die ... gebracht. Fu00FCr die Abholung ist neben der Benachrichtigung die Vorlage Ihres Ausweises sowie ggf. eine Vollmacht erforderlich." 
df.loc[(df['status_next'].str.startswith("Die Sendung wurde zur Abholung in die")) & (df['status_next'].str.endswith("gebracht.")), "status_next"] = "Die Sendung wurde zur Abholung in die ... gebracht." 
df.loc[(df['status_next'].str.startswith("Die Zustellung am gewu00FCnschten Liefertag")) & (df['status_next'].str.endswith("wurde gewu00E4hlt.")), "status_next"] = "Die Zustellung am gewu00FCnschten Liefertag ... wurde gewu00E4hlt." 

In [ ]:
pd.DataFrame(df["status"].value_counts()).head(10)

In [ ]:
pd.DataFrame(df["status"].value_counts()).tail(10)

In [ ]:
len(df["status"].unique())

In [ ]:
len(df["status_next"].unique())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.xticks(rotation='vertical')
df["status"].hist(figsize=(15,5))

We introduces two new states to status_next: SuccessDelivery, FailedDelivery

### Distribution of the locations

In [ ]:
len(df["ort"].unique())

In [ ]:
pd.DataFrame(df["ort"].value_counts()).head(20)

In [ ]:
pd.DataFrame(df["ort"].value_counts()).tail(20)

Notice: One idea here could be to merge some locations. 

### Remove unused columns

In [ ]:
df.drop(columns=["crawltime", "url"], inplace=True)

In [ ]:
df.head()

### Distribution of event times

In [ ]:
counts = df['datum'].dt.dayofweek.value_counts(sort=False)
plt.bar(counts.index,counts)
plt.show()

We see in the above plot which days of the week have the highest amount of events. Sunday has for example the least amount of events, as expected.

In [ ]:
counts = df['datum'].dt.hour.value_counts(sort=False)
plt.bar(counts.index,counts)
plt.show()

In the above plot we see the distribution of the events over 24 hours of a day

### Time diff between events

In [ ]:
df['datum'] = pd.to_datetime(df['datum'], utc=True)
df['datum_next'] = pd.to_datetime(df['datum_next'], utc=True)
df["datum_diff"] = df["datum_next"] - df["datum"]

Notice that for is_final_event=1, the datum_diff does not make any sense. Do not use it in the analysis

In [ ]:
df[df["is_final_event"] == 0]['datum_diff'].astype('timedelta64[h]').plot.hist()

We observe two interesting things here. There are events with zero time diff and there are events with very big time diffs.

### Events with very small time diffs

In [ ]:
df[(df["is_final_event"] == 0.0) & (df['datum_diff'].astype('timedelta64[s]') == 0)] 

### Events with huge time diffs

In [ ]:
df[(df["is_final_event"] == 0.0) & (df['datum_diff'].astype('timedelta64[h]') > 150)] 

### Distribution of time diff for various states

In [ ]:
df["datum_diff_h"] = df['datum_diff'].astype('timedelta64[h]')

In [ ]:
df_temp = df[df["is_final_event"] == 0][["datum_diff_h", "status"]]
df_temp.reset_index()
df_temp.index = range(0,df_temp.shape[0])
df_temp.head()

In [ ]:
%matplotlib inline
df_temp["datum_diff_h"].hist(by=df_temp['status'], layout=(len(df_temp["status"].unique()), 1), figsize=(10,80))
plt.tight_layout()
df.drop(columns=["datum_diff_h"], inplace=True)

In [ ]:
df.head()

### Distribution of time diff for various locations

In [ ]:
df["datum_diff_h"] = df['datum_diff'].astype('timedelta64[h]')

In [ ]:
df_temp = df[df["is_final_event"] == 0][["datum_diff_h", "status", "ort"]]
df_temp.reset_index()
df_temp.index = range(0,df_temp.shape[0])
df_temp.head()

In [ ]:
%matplotlib inline
df_temp["datum_diff_h"].hist(by=df_temp['ort'], layout=(len(df_temp["ort"].unique()), 1), figsize=(10,140))
plt.tight_layout()
df.drop(columns=["datum_diff_h"], inplace=True)

### Distribution of time diff for various locations and states

In [ ]:
df["datum_diff_h"] = df['datum_diff'].astype('timedelta64[h]')

In [ ]:
df_temp = df[df["is_final_event"] == 0][["datum_diff_h", "status", "ort"]]
df_temp.reset_index()
df_temp.index = range(0,df_temp.shape[0])
df_temp.head()

In [ ]:
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline


grouped = df_temp.groupby(["status", "ort"])

for name, group in grouped:
    fig = plt.figure()
    fig.suptitle(str(name) + ": #{}".format(len(group)))
    matplotlib.pyplot.hist(group.datum_diff_h)
    plt.show()
    
df.drop(columns=["datum_diff_h"], inplace=True)

In [ ]:
df.head()